In [ ]:
import numpy as np
import section4_7_h as appendix_H

## 付録B　ガス温水暖房機及びガス給湯温水暖房機 / Appendix B Gas water heater and gas hot water heater

### B.1 記号及び単位 / Symbol and unit

省略

ommision

### B.2 エネルギー消費量 / Energy consumption

#### B.2.1 ガス消費量 / Gas consumption

$$\large
E_{G,hs} = \left\{ \begin{array}{ll}
    0&(Q_{out,H,hs} = 0)\\ 
    \frac{Q_{out,H,hs} + Q_{body}}{e_{ex}} &(Q_{out,H,hs} > 0)\\ 
    \end{array}
\right.
$$

$E_{G,hs}$：1 時間当たりの温水暖房用熱源機のガス消費量(MJ/h) / Gas consumption of hot water heating heat source machine per hour  
$Q_{out,H,hs}$：1 時間当たりの温水暖房用熱源機の暖房出力(MJ/h) / Heating output of the heat source machine for hot water heating per hour  
$Q_{body}$：1 時間当たりの温水暖房用熱源機の筐体放熱損失(MJ/h) / Heat dissipation loss of the case of the heat source unit for hot water heating per hour  
$e_{ex}$：1 時間平均の温水暖房用熱源機の熱交換効率 / Heat exchange efficiency of one hour average hot water heating heat source machine

In [ ]:
# 1時間当たりの温水床暖房用熱源機のガス消費量 (1)
def get_E_G_hs(e_rtd, q_rtd_hs, Q_out_H_hs, hs_type, P_hs):
    """
    :param e_rtd: 温水暖房用熱源機の定格効率
    :param q_rtd_hs: 温水暖房用熱源機の定格能力 (W)
    :param Q_out_H_hs: 1時間当たりの温水暖房用熱源機の暖房出力 (MJ/h)
    :param hs_type: 温水暖房用熱源機の種類
    :param P_hs: 送水温度の区分
    :return: 1時間当たりの温水床暖房用熱源機のガス消費量 (MJ/h)
    """
    # 1時間当たりの温水暖房用熱源機の筐体放熱損失 (MJ/h) (2)
    Q_body = get_Q_body(hs_type, P_hs)

    # 熱交換効率
    e_ex = get_e_ex(e_rtd, Q_body, hs_type, P_hs, q_rtd_hs)

    # 1時間当たりの温水床暖房用熱源機のガス消費量 (1)
    E_G_hs = (Q_out_H_hs + Q_body) / e_ex
    E_G_hs[Q_out_H_hs == 0] = 0

    return E_G_hs

当該住戸のガス温水暖房機がガス従来型温水暖房機の場合

When the gas hot water heater of the dwelling unit is a gas conventional hot water heater

$$\large
Q_{body} = 240.96 \times 3600 \times 10^{-6}
$$

当該住戸のガス温水暖房機がガス潜熱回収型温水暖房機の場合でかつ送水温度の区分$P_{hs}$が 1（送水温度60℃）の場合

When the gas hot water heater of the dwelling unit concerned is a gas latent heat recovery type hot water heater and the division $P_{hs}$ of water supply temperature is 1 (water supply temperature 60 ° C)

$$\large
Q_{body} = 225.26 \times 3600 \times 10^{-6}
$$

当該住戸のガス温水暖房機がガス潜熱回収型温水暖房機の場合でかつ送水温度の区分$P_{hs}$が 2（送水温度40℃）の場合

When the gas hot water heater of the dwelling unit concerned is a gas latent heat recovery type hot water heater and the division $P_{hs}$ of water supply temperature is 2 (water supply temperature 40 ° C)

$$\large
Q_{body} = 123.74 \times 3600 \times 10^{-6}
$$

In [ ]:
# 1時間当たりの温水暖房用熱源機の筐体放熱損失 (2)
def get_Q_body(hs_type, P_hs):
    """
    :param hs_type: 温水暖房用熱源機の種類
    :param P_hs: 送水温度の区分
    :return: 1時間当たりの温水暖房用熱源機の筐体放熱損失 (MJ/h)
    """
    if hs_type in ['ガス従来型温水暖房機', 'ガス従来型給湯温水暖房機', 'ガス従来型'] or hs_type == '不明':
        # (2a)
        Q_body = np.ones(24*365) * 240.96 * 3600 * 10 ** (-6)
        return Q_body
    elif hs_type in ['ガス潜熱回収型温水暖房機', 'ガス潜熱回収型給湯温水暖房機', 'ガス潜熱回収型']:
        Q_body = np.zeros(24*365)

        # (2b-1)
        Q_body[P_hs == 1] = 225.26 * 3600 * 10 ** (-6)

        # (2b-2)
        Q_body[P_hs == 2] = 123.74 * 3600 * 10 ** (-6)

        return Q_body
    else:
        raise ValueError(hs_type)

$$\large
e_{ex,d,t} = e_{rtd} \times f_{rtd} \times \frac{q_{rtd,hs} \times 3600 \times 10^{-6} + Q_{body,d,t}}{q_{rtd,hs} \times 3600 \times 10^{-6}}
$$

$e_{rtd}$：温水暖房用熱源機の定格効率 / Rated efficiency of the hot water heating heat source machine  
$f_{rtd}$：1 時間平均の定格効率を補正する係数 / Factor to correct rated efficiency of 1 hour average  
$q_{rtd,hs}$：温水暖房用熱源機の定格能力(W) / Rated capacity of hot water heating heat source machine  

In [ ]:
# 1時間平均の温水暖房用熱源機の熱交換効率 (3)
def get_e_ex(e_rtd, Q_body, hs_type, P_hs, q_rtd_hs):
    """
    :param e_rtd: 温水暖房用熱源機の定格効率 (-)
    :param Q_body: 1時間当たりの温水暖房用熱源機の筐体放熱損失 (MJ/h)
    :param hs_type: 温水暖房用熱源機の種類
    :param P_hs: 送水温度の区分
    :param q_rtd_hs: 温水暖房用熱源機の定格能力 (W)
    :return: 1時間平均の温水暖房用熱源機の熱交換効率
    """
    # 定格効率を補正する係数
    f_rtd = get_f_rtd(hs_type, P_hs)

    return e_rtd * f_rtd * (q_rtd_hs * 3600 * 10 ** (-6) + Q_body) / (q_rtd_hs * 3600 * 10 ** (-6))

定格効率を補正する係数$f_{rtd}$は、当該住戸のガス温水暖房機が従来型ガス温水暖房機の場合0.985とし、当該住戸のガス温水暖房機が潜熱回収型ガス温水暖房機の場合でかつ送水温度の区分$P_{hs}$が 1（送水温度 60℃）の場合、1.038とし、当該住戸のガス温水暖房機が潜熱回収型ガス温水暖房機の場合でかつ送水温度の区分$P_{hs}$が 2（送水温度 40℃）の場合、1.064とする。 

The coefficient $f_{rtd}$ to correct the rated efficiency is 0.985 when the gas hot water heater of the dwelling unit is the conventional gas hot water heater, and the division $P_{hs}$ of the water temperature when the gas hot water heater of the dwelling unit is the latent heat recovery type gas hot water heater If the value is 1 (Water supply temperature 60 ° C), it shall be 1.038, and if the gas water heater of the dwelling unit is a latent heat recovery type gas water heater and the division $P_{hs}$ of the water supply temperature is 2 (Water supply temperature 40 ° C), it shall be 1.064.

In [ ]:
# 定格効率を補正する係数
def get_f_rtd(hs_type, P_hs):
    """
    :param hs_type: 温水暖房用熱源機の種類
    :param P_hs: 送水温度の区分
    :return: 定格効率を補正する係数
    """
    if hs_type in ['ガス従来型温水暖房機', 'ガス従来型給湯温水暖房機', 'ガス従来型']:
        return np.ones(24*365) * 0.985
    elif hs_type in ['ガス潜熱回収型温水暖房機', 'ガス潜熱回収型給湯温水暖房機', 'ガス潜熱回収型']:
        f_rtd = np.zeros(24*365)
        f_rtd[P_hs == 1] = 1.038
        f_rtd[P_hs == 2] = 1.064
        return f_rtd
    else:
        raise ValueError(hs_type)

#### B.2.2 消費電力量 / Power consumption

$$\large
E_{E,hs} = E_{E,hs,pmp} + E_{E,hs,fan}
$$

$E_{E,hs}$：1 時間当たりの温水暖房用熱源機の消費電力量(kWh/h) / Power consumption of the hot water heating heat source unit per hour  
$E_{E,hs,pmp}$：1 時間当たりの温水暖房用熱源機の送水ポンプの消費電力量(kWh/h) / Power consumption of the water supply pump of the heat source unit for hot water heating per hour  
$E_{E,hs,fan}$：1 時間当たりの温水暖房用熱源機の排気ファンの消費電力量(kWh/h) / Power consumption of exhaust fan of heat source unit for hot water heating per hour  

In [ ]:
# 消費電力量 (4)
def get_E_E_hs(r_WS_hs, E_G_hs):
    # 送水ポンプの消費電力量
    E_E_hs_pmp = get_E_E_hs_pmp(r_WS_hs)

    # 排気ファンの消費電力量
    E_E_hs_fan = get_E_E_hs_fan(E_G_hs)

    return E_E_hs_pmp + E_E_hs_fan

$$\large
E_{E,hs,pmp} = P_{hs,pmp} \times r_{WS,hs} \times 10^{-3}
$$

$P_{hs,pmp}$：温水暖房用熱源機の送水ポンプの消費電力(W) / Power consumption of water pump of heat source machine for hot water heating  
$r_{WS,hs}$：温水暖房用熱源機の温水供給運転率 / Hot water supply operation rate of heat source equipment for hot water heating

温水暖房用熱源機の送水ポンプの消費電力$P_{hs,pmp}$は、73(W)とする。 

The power consumption $ P_{hs, pmp} $ of the water pump of the hot water heating heat source machine is 73 (W).

In [ ]:
# 送水ポンプの消費電力量 (5)
def get_E_E_hs_pmp(r_WS_hs):
    # 送水ポンプの消費電力
    P_hs_pmp = get_P_hs_pmp()

    return P_hs_pmp * r_WS_hs * 10 ** (-3)


# 送水ポンプの消費電力
def get_P_hs_pmp():
    return 73

$$\large
E_{E,hs,fan} = E_{G,hs} \times \gamma \times 10^{3} \div 3600
$$

$E_{G,hs}$：1 時間当たりの温水暖房用熱源機のガス消費量(MJ/h) / Gas consumption of hot water heating heat source machine per hour  
$\gamma$：排気ファンの効率 / Exhaust fan efficiency  

In [ ]:
# 排気ファンの消費電力量 (6)
def get_E_E_hs_fan(E_G_hs):
    # 排気ファンの効率
    gamma = get_gamma()

    return E_G_hs * gamma * 10 ** 3 / 3600


# 排気ファンの効率
def get_gamma():
    return 0.003

#### B.2.3 灯油消費量 / Kerosene consumption

1 時間当たりの温水暖房用熱源機の灯油消費量$E_{K,hs}$は0とする。 

The kerosene consumption $ E_{K, hs} $ of the heat source equipment for hot water heating per hour is 0.

In [ ]:
# 1時間当たりの温水暖房用熱源機の灯油消費量
def get_E_K_hs():
    # 1時間当たりの温水暖房用熱源機の灯油消費量は0とする
    return np.zeros(24 * 365)

#### B.2.4 その他の一次エネルギー消費量 / Other primary energy consumption

1 時間当たりの温水暖房用熱源機のその他の一次エネルギー消費量$E_{M,hs}$は0とする。 

The other primary energy consumption $ E_{M, hs} $ of the hot water heating heat source unit per hour is assumed to be zero.

In [ ]:
# 1時間当たりの温水暖房用熱源機のその他の一次エネルギー消費量
def get_E_M_hs():
    # 1時間当たりの温水暖房用熱源機のその他の一次エネルギー消費量は0とする
    return np.zeros(24 * 365)

### B.3 最大暖房出力 / Maximum heating output

$$\large
Q_{max,H,hs} = q_{rtd,hs} \times 3600 \div 10^{6} 
$$

In [ ]:
# 1時間当たりの温水暖房用熱源機の最大暖房出力 (7)
def get_Q_max_H_hs(q_rtd_hs):
    """
    :param q_rtd_hs: 温水暖房用熱源機の定格能力 (W)
    :return: 1時間当たりの温水暖房用熱源機の最大暖房出力 (MJ/h)
    """
    return np.ones(24*365) * q_rtd_hs * 3600 / (10 ** 6)